In [1]:
! nvidia-smi

Fri Jun  6 16:18:30 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.247.01             Driver Version: 535.247.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:35:00.0 Off |                    0 |
| N/A   39C    P8              16W /  72W |      0MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
! pip install huggingface_hub
! pip install -U vllm transformers accelerate
! pip install auto-gptq
! pip install optimum
! pip install llmcompressor

In [ ]:
from vllm import LLM, SamplingParams
import torch
from huggingface_hub import login, snapshot_download
import json


torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

login(token="addHuggingFace token")

INFO 06-06 16:18:45 [__init__.py:243] Automatically detected platform cuda.


<h3><b>Download the non-supported model from huggingface</b></h3>

In [4]:
model_name = "unsloth/Qwen2.5-0.5B-Instruct"
# "meta-llama/Llama-3.2-1B-Instruct" 
# "Qwen/Qwen2.5-1.5B-Instruct-AWQ"  *** 
# "Qwen/Qwen3-1.7B-GPTQ-Int8"   *** -> use reasoning 
# "Qwen/Qwen2.5-1.5B-Instruct-GPTQ-Int4" ***** 
# "Qwen/Qwen2.5-1.5B-Instruct-GPTQ-Int8" # "Qwen/Qwen2.5-1.5B" 
# "Qwen/Qwen2-VL-2B-Instruct" 
# "Qwen/Qwen2.5-0.5B" 
#"Qwen/Qwen2.5-3B-Instruct" 
llm = LLM(model=model_name, dtype="auto")


INFO 06-06 16:18:48 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-06 16:18:48 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-06 16:18:48 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.


config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

INFO 06-06 16:18:57 [config.py:793] This model supports multiple tasks: {'score', 'reward', 'classify', 'embed', 'generate'}. Defaulting to 'generate'.
INFO 06-06 16:18:57 [config.py:2118] Chunked prefill is enabled with max_num_batched_tokens=8192.


tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

WARNING 06-06 16:18:59 [utils.py:2531] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 06-06 16:19:03 [__init__.py:243] Automatically detected platform cuda.
INFO 06-06 16:19:06 [core.py:438] Waiting for init message from front-end.
INFO 06-06 16:19:06 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-06 16:19:06 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-06 16:19:06 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.
INFO 06-06 16:19:06 [core.py:65] Initializing a V1 LLM engine (v0.9.0.1) with config: model='unsloth/Qwen2.5-0.5B-Instruct', speculative_config=None, tokenizer='unsloth/Qwen2.5-0.5B-Instruct', skip_tokenize

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  6.79it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  6.78it/s]



INFO 06-06 16:19:09 [default_loader.py:280] Loading weights took 0.18 seconds
INFO 06-06 16:19:09 [gpu_model_runner.py:1549] Model loading took 0.9269 GiB and 2.057077 seconds
INFO 06-06 16:19:15 [backends.py:459] Using cache directory: /home/zeus/.cache/vllm/torch_compile_cache/76e57c5318/rank_0_0 for vLLM's torch.compile
INFO 06-06 16:19:15 [backends.py:469] Dynamo bytecode transform time: 5.66 s
INFO 06-06 16:19:19 [backends.py:132] Directly load the compiled graph(s) for shape None from the cache, took 3.858 s
INFO 06-06 16:19:20 [monitor.py:33] torch.compile takes 5.66 s in total
INFO 06-06 16:19:21 [kv_cache_utils.py:637] GPU KV cache size: 1,515,440 tokens
INFO 06-06 16:19:21 [kv_cache_utils.py:640] Maximum concurrency for 32,768 tokens per request: 46.25x
INFO 06-06 16:19:41 [gpu_model_runner.py:1933] Graph capturing finished in 20 secs, took 1.23 GiB
INFO 06-06 16:19:41 [core.py:167] init engine (profile, create kv cache, warmup model) took 31.50 seconds


In [5]:
class ModelCall(): 
    def __init__(self, temperature:float=0.65, top_p:float=0.95, max_tokens:int=512):
        self.temperature=temperature
        self.top_p=top_p
        self.max_tokens=max_tokens

    def makeQuery(self, query): 
        from vllm import LLM, SamplingParams
        self.sampling_params = SamplingParams(temperature=self.temperature, top_p=self.top_p, max_tokens=self.max_tokens)
        if not any(word in query.lower() for word in ["email", "mail", "send an email", "write an email", "compose email"]):
            self.messages = [
                {
                    "role": "system",
                    "content": (
                        "You are a strict assistant that only helps with writing emails. "
                        "If a user gives a prompt that is not related to writing an email, "
                        "you will ask them to provide a clear instruction to write an email."
                    )
                },
                {
                    "role": "user",
                    "content": query
                }
            ]
        else:
            self.messages = [
                {
                    "role": "system",
                    "content": (
                        "You are a helpful email writing assistant. "
                        "You are going to write an email based on the user's prompt. "
                        "You will write the email in a professional tone."
                    )
                },
                {
                    "role": "user",
                    "content": query
                }
            ]
    def retQuery(self):
        outputs = llm.chat(self.messages, self.sampling_params)
        return outputs[0].outputs[0].text       


In [11]:
llmQNA = ModelCall()
# llmQNA.makeQuery("Make an Email to CEO to make a annual tour in Finland at the end of the year")
llmQNA.makeQuery("Hi, hpow are you? give me example to 3 newton phythocs law")
print(llmQNA.retQuery())

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Hello! I'm sorry, but I can't assist with that.


In [ ]:
! nvidia-smi

<h2><b>Now apply NGRok to process the communication from outside</b></h3>

In [ ]:
!pkill -f ngrok
!pkill -f flask
!wget -q -O ngrok.zip https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
!unzip -o ngrok.zip
!chmod +x ngrok
! pip install flask

<h3><b>Setup ngrik tokens</b></h3>

In [ ]:
NGROK_AUTH_TOKEN = "2xyuXqhmEs7i5RGY7bCJPLokKa3_soAZjNfU4fbTY2ZEeDV8"
!./ngrok authtoken $NGROK_AUTH_TOKEN

<h3><b>Build Flask Server</b></h3>

In [ ]:
from flask import Flask, request, jsonify
from threading import Thread
import subprocess
import time
import requests

app = Flask(__name__)

@app.route('/')
def index():
    return "Flask app is running!"

@app.route('/ask', methods=['POST'])
def ask():
    data = request.get_json()
    question = data.get("question", "")

    llmQNA = ModelCall()
    llmQNA.makeQuery("Make an Email to CEO to make a annual tour in Finland at the end of the year")
    output = llmQNA.retQuery()

    return jsonify({"answer": output})

<h3><b>Start Flask Server</b></h3>

In [10]:
def run_flask():
    app.run(host="0.0.0.0", port=5471)
Thread(target=run_flask).start()

<h3><b>Start ngrok Server</b></h3>

In [11]:
time.sleep(2)  
ngrok_process = subprocess.Popen(["./ngrok", "http", "5471"])

<h3><b>Get ngrok https link</b></h3>

In [ ]:
def get_ngrok_url():
    for _ in range(15):
        try:
            res = requests.get("http://localhost:4040/api/tunnels")
            tunnels = res.json().get("tunnels")
            if tunnels:
                return tunnels[0]["public_url"]
        except Exception:
            pass
        time.sleep(1)
    return None

public_url = get_ngrok_url()
if public_url:
    print("Public URL:", public_url)
else:
    print("Failed to fetch ngrok URL after waiting")

In [ ]:
!curl http://localhost:4040/api/tunnels